In [7]:
import os
import winsound         # for sound  
import win32com.client
import pythoncom
import datetime, time# for sleep
import sys
import os.path
from os import path
import pandas as pd # command 로 시작할 때는 모듈 import 위치고려
from datetime import timedelta
import codecs
import shutil
from init_Stock import *

def __StandbyCall():
#     start_time = "09:00"
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Login_History"+_Name+".txt"
    print("# Standby Start : %s" % (datetime.datetime.today().strftime("%H:%M:%S")))
    while datetime.datetime.today().strftime("%H:%M") < start_time : # ---> start_time보다 크면 while문 빠져나감, 시작 Call
        now_time = datetime.datetime.today().strftime("%H:%M:%S")
        
        if now_time > "08:55:00" : # 8시 55분보다 늦은 시간이면 print를 찍는다
            print("# Ready to Start ..... now %s" % (now_time))
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            output.write("# Ready to Start ..... now %s\n" % (now_time))
            output.close() # ---> close 가 되어야 write 처리가 완료된다
            
        time.sleep(60)
        
    print("\n# Standby End : %s" % (datetime.datetime.today().strftime("%H:%M:%S")))

def __Init_Conn() :

    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Login_History"+_Name+".txt"
    class XASessionEventHandler:
        
        login_state = 0

        def OnLogin(self, code, msg):

            if code == "0000":
                print("로그인 성공")
                # output = open(File_Name, "a")
                output = codecs.open(File_Name, "a", "utf-8-sig") 
                output.write("# %s ---> 로그인 성공 \n" % (datetime.datetime.today().strftime("%Y%m%d %H%M")))
                output.close() # ---> close 가 되어야 write 처리가 완료된다
                XASessionEventHandler.login_state = 1
            else:
                print("로그인 실패")
                # output = open(File_Name, "a")
                output = codecs.open(File_Name, "a", "utf-8-sig") 
                output.write("# %s ---> 로그인 에러 \n" % (datetime.datetime.today().strftime("%Y%m%d %H%M")))
                output.close() # ---> close 가 되어야 write 처리가 완료된다

    instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEventHandler)

    # 접속할 서버의 기본 주소는 'hts.ebestsec.co.kr'인데 모의 투자인 경우에는 'demo.ebestsec.co.kr'을 사용
    print("Server_Name : %s" % Server_Name)
    instXASession.ConnectServer(Server_Name, 20001)
#     instXASession.ConnectServer("hts.ebestsec.co.kr", 20001)
    instXASession.Login(id, passwd, cert_passwd, 0, 0)

    while XASessionEventHandler.login_state == 0:
        pythoncom.PumpWaitingMessages()

    num_account = instXASession.GetAccountListCount()
#     for i in range(num_account):
#         account = instXASession.GetAccountList(i)
#         print("계좌번호 : %s" % (account))

def __NAV_History(expcode) :

    Long_Term_Dic = {}

    ##################################################
    # 주식 현재가 호가 조회
    
    class XAQueryEventHandlerT1903:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT1903.query_state = 1

        def OnReceiveMessage(self, error, code, message):
            print("OnreceiveMessage", error, code, message)

    ##################################################
    instXAQueryT1903 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT1903)
    instXAQueryT1903.ResFileName = "C:\\eBEST\\xingAPI\\Res\\T1903.res"
    instXAQueryT1903.SetFieldData("T1903InBlock", "shcode", 0, expcode)
    # instXAQueryT1903.SetFieldData("T1903InBlock", "date", 0, 20210511)

    instXAQueryT1903.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT1903.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()

    count = instXAQueryT1903.GetBlockCount("t1903OutBlock1")

    # print(count)
    # date_list, open_list, close_list, change_list, diff_list = [], [], [], [], []
    for i in range(count):
        date = instXAQueryT1903.GetFieldData("T1903OutBlock1", "date", i)
        price = instXAQueryT1903.GetFieldData("T1903OutBlock1", "price", i) # 현재가
        sign = instXAQueryT1903.GetFieldData("T1903OutBlock1", "sign", i) # 전일대비 구분
        change = instXAQueryT1903.GetFieldData("T1903OutBlock1", "change", i) # 전일대비
        volume = instXAQueryT1903.GetFieldData("T1903OutBlock1", "volume", i) # 누적거래량
        navdiff = instXAQueryT1903.GetFieldData("T1903OutBlock1", "navdiff", i) # NAV대비
        nav = instXAQueryT1903.GetFieldData("T1903OutBlock1", "nav", i) # NAV ---> 현재 자산가치
        navchange = instXAQueryT1903.GetFieldData("T1903OutBlock1", "navchange", i) # 전일대비
        crate = instXAQueryT1903.GetFieldData("T1903OutBlock1", "crate", i) # 추적오차
        grate = instXAQueryT1903.GetFieldData("T1903OutBlock1", "grate", i) # 괴리
        jisu = instXAQueryT1903.GetFieldData("T1903OutBlock1", "jisu", i) # 지구
        jichange = instXAQueryT1903.GetFieldData("T1903OutBlock1", "jichange", i) # 전일대비
        jirate = instXAQueryT1903.GetFieldData("T1903OutBlock1", "jirate", i) # 전일대비율

        # print(date, price, sign, change, volume, navdiff, navchange, crate, grate, jisu, jichange, jirate)
        nav = float(nav)
        price = float(price)
        # print(price, nav, ((price-nav)/price)*100)
        # print(price, nav, ((price-nav)/nav)*100)
        print(price, nav, grate)

        # diff_list.append(diff)

def __NAV_Current(expcode) :

    Long_Term_Dic = {}

    ##################################################
    # 주식 현재가 호가 조회
    
    print(expcode)
    print("# 1")
    class XAQueryEventHandlerT1901:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT1901.query_state = 1

        def OnReceiveMessage(self, error, code, message):
            print("OnreceiveMessage", error, code, message)

    ##################################################
    instXAQueryT1901 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT1901)
    instXAQueryT1901.ResFileName = "C:\\eBEST\\xingAPI\\Res\\T1901.res"
    instXAQueryT1901.SetFieldData("T1901InBlock", "shcode", 0, expcode)

    instXAQueryT1901.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송
    print("# 2")

    while XAQueryEventHandlerT1901.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()

    count = instXAQueryT1901.GetBlockCount("t1901OutBlock")

    print("# 3")
    print(count)
    # date_list, open_list, close_list, change_list, diff_list = [], [], [], [], []
    for i in range(count):
        print("# for")
        hname = instXAQueryT1901.GetFieldData("T1901OutBlock", "hname", i)
        price = instXAQueryT1901.GetFieldData("T1901OutBlock", "price", i) # 현재가
        sign = instXAQueryT1901.GetFieldData("T1901OutBlock", "sign", i) # 전일대비 구분
        change = instXAQueryT1901.GetFieldData("T1901OutBlock", "change", i) # 전일대비
        diff = instXAQueryT1901.GetFieldData("T1901OutBlock", "diff", i) # 등락율
        volume = instXAQueryT1901.GetFieldData("T1901OutBlock", "volume", i) # 누적거래량
        recprice = instXAQueryT1901.GetFieldData("T1901OutBlock", "recprice", i) # 기준가
        high52w = instXAQueryT1901.GetFieldData("T1901OutBlock", "high52w", i) # 52주 최고가
        low52w = instXAQueryT1901.GetFieldData("T1901OutBlock", "low52w", i) # 52주 최저가
        per = instXAQueryT1901.GetFieldData("T1901OutBlock", "per", i) # 52주 최저가
        nav = instXAQueryT1901.GetFieldData("T1901OutBlock", "nav", i) # NAV ---> 현재 자산가치
        navdiff = instXAQueryT1901.GetFieldData("T1901OutBlock", "navdiff", i) # NAV대비
        navchange = instXAQueryT1901.GetFieldData("T1901OutBlock", "navchange", i) # 전일대비
        cocrate = instXAQueryT1901.GetFieldData("T1901OutBlock", "cocrate", i) # 추적오차율
        kasis = instXAQueryT1901.GetFieldData("T1901OutBlock", "kasis", i) # 괴리율

        print(hname, price, sign, change, diff, volume, recprice, high52w, low52w, per, nav, navdiff, navchange, cocrate, kasis, nav, price)

        # diff_list.append(diff)
    print("# 4")

if __name__ == '__main__' :
    
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    # File_Name = "D:\\Python\\Log\\Main_Status_History"+_Name+".txt"

    print("# Init Connection")
    __Init_Conn()

    expcode = "233740"
    __NAV_History(expcode)
    __NAV_Current(expcode)

# Init Connection
Server_Name : hts.ebestsec.co.kr
로그인 성공
OnreceiveMessage 0 00000 조회완료
13460.0 13531.13 -0.53
13810.0 13901.24 -0.66
14485.0 14539.89 -0.38
13910.0 13997.53 -0.63
13550.0 13601.72 -0.38
13615.0 13728.11 -0.82
13405.0 13474.96 -0.52
14200.0 14336.0 -0.95
14385.0 14540.56 -1.07
14765.0 14925.58 -1.08
15670.0 15826.16 -0.99
16075.0 16282.4 -1.27
16010.0 16189.51 -1.11
16010.0 16208.74 -1.23
15950.0 16121.77 -1.07
16390.0 16539.32 -0.90
16540.0 16749.78 -1.25
16150.0 16331.36 -1.11
15920.0 16097.8 -1.10
15920.0 16141.57 -1.37
233740
# 1
# 2
OnreceiveMessage 0 00000 조회완료
# 3
1
# for
KODEX 코스닥150 레버 13460 5 350 -2.53 12525648 13810 18030 8410 0.00 13531.13 -2.66 370.11 1.26 0.00 13531.13 13460
# 4
